In [3]:
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox
from PIL import Image, ImageTk, ImageOps, ImageEnhance,ImageFilter
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model 

# Global variables to hold image data and cropping coordinates
img = None
original_img = None
img_display = None
crop_coords = None
rect_id = None
selected_color = None

# Constants for display size
LARGE_DISPLAY_WIDTH = 400 
LARGE_DISPLAY_HEIGHT = 500  


# Define display dimensions globally for access in functions
DISPLAY_WIDTH = LARGE_DISPLAY_WIDTH 
DISPLAY_HEIGHT = LARGE_DISPLAY_HEIGHT

# Load the pre-trained U-Net model 
model = load_model('unet_model2.keras')


# Global variable to store the current image
img_display = None

def open_image():
    global img, original_img, img_display
    file_path = filedialog.askopenfilename(filetypes=[("Image Files", "*.jpg;*.jpeg;*.png")])
    if file_path:
        img = Image.open(file_path)
        original_img = img.copy()
        img_display = img.copy()
        update_canvas_sizes()
        show_images()

def show_images():
    """Displays both the original and the modified images."""
    if original_img and img_display:
        # Resize images to fit their respective canvases
        original_resized = original_img.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
        img_original_tk = ImageTk.PhotoImage(original_resized)
        canvas_original.create_image(0, 0, anchor="nw", image=img_original_tk)
        canvas_original.img_original_tk = img_original_tk

        modified_resized = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
        img_modified_tk = ImageTk.PhotoImage(modified_resized)
        canvas_updated.create_image(0, 0, anchor="nw", image=img_modified_tk)
        canvas_updated.img_modified_tk = img_modified_tk

def update_canvas_sizes():
    """Adjust the canvas sizes based on the fixed display dimensions."""
    canvas_original.config(width=LARGE_DISPLAY_WIDTH, height=LARGE_DISPLAY_HEIGHT)
    canvas_updated.config(width=LARGE_DISPLAY_WIDTH, height=LARGE_DISPLAY_HEIGHT)

def reset_image():
    global img_display
    img_display = original_img.copy()
    img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
    update_canvas_sizes()
    show_images()

def start_crop(event):
    global crop_coords, rect_id
    crop_coords = [event.x, event.y, event.x, event.y]
    rect_id = canvas_updated.create_rectangle(*crop_coords, outline='red')

def update_crop(event):
    global crop_coords
    if crop_coords:
        crop_coords[2], crop_coords[3] = event.x, event.y
        canvas_updated.coords(rect_id, *crop_coords)

def end_crop(event):
    global img_display, crop_coords
    if crop_coords and img_display:
        x_start, y_start, x_end, y_end = crop_coords
        x_start, x_end = sorted([x_start, x_end])
        y_start, y_end = sorted([y_start, y_end])
        # Calculate the actual position in the image
        actual_x_start = int(x_start * img_display.width / LARGE_DISPLAY_WIDTH)
        actual_x_end = int(x_end * img_display.width / LARGE_DISPLAY_WIDTH)
        actual_y_start = int(y_start * img_display.height / LARGE_DISPLAY_HEIGHT)
        actual_y_end = int(y_end * img_display.height / LARGE_DISPLAY_HEIGHT)
        img_display = img_display.crop((actual_x_start, actual_y_start, actual_x_end, actual_y_end))
        img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
        update_canvas_sizes()
        show_images()
        crop_coords = None
        canvas_updated.delete(rect_id)

def convert_color(option):
    global img_display
    if img_display:
        if option == 'Grayscale':
            img_display = ImageOps.grayscale(img_display)
        elif option == 'Black & White':
            img_display = img_display.convert('L').point(lambda x: 0 if x < 128 else 255, '1')
        else:
            img_display = original_img.copy()
        img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
        update_canvas_sizes()
        show_images()

def rotate_image():
    global img_display
    if img_display:
        angle = simpledialog.askfloat("Rotate", "Enter angle:", minvalue=0, maxvalue=360)
        if angle is not None:
            img_display = img_display.rotate(angle, expand=True)
            img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
            update_canvas_sizes()
            show_images()

def flip_image(flip_type):
    global img_display
    if img_display:
        if flip_type == 'Horizontal':
            img_display = img_display.transpose(Image.FLIP_LEFT_RIGHT)
        elif flip_type == 'Vertical':
            img_display = img_display.transpose(Image.FLIP_TOP_BOTTOM)
        img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)
        update_canvas_sizes()
        show_images()

def save_image():
    if img_display:
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("JPEG files", "*.jpg"), ("All files", "*.*")])
        if file_path:
            img_display.save(file_path)

def pick_color(event):
    """Function to pick a color from the image where the user clicks."""
    global selected_color
    if img_display:
        # Calculate the actual position in the image
        actual_x = int(event.x * img_display.width / DISPLAY_WIDTH)
        actual_y = int(event.y * img_display.height / DISPLAY_HEIGHT)
        if 0 <= actual_x < img_display.width and 0 <= actual_y < img_display.height:
            rgb_image = img_display.convert('RGB')  # Ensure image is in RGB mode
            selected_color = rgb_image.getpixel((actual_x, actual_y))
            messagebox.showinfo("Color Selected", f"Selected Color: {selected_color}")
            # Call segmentation after selecting the color
            segment_image()

def segment_image():
    """Segment the image based on the color selected by the user."""
    global img_display, selected_color
    if img_display and selected_color:
        # Ensure the image is in RGB mode for consistency
        if img_display.mode != 'RGB':
            img_display = img_display.convert('RGB')

        img_np = np.array(img_display)  # Convert the image to a NumPy array
        target_color = np.array(selected_color)  # Convert selected color to a NumPy array

        # Calculate the color difference using Euclidean distance
        diff = np.linalg.norm(img_np - target_color, axis=-1)  # Calculate the color difference

        # Ask the user for a threshold
        threshold = simpledialog.askinteger("Segmentation Threshold", "Enter color distance threshold (e.g., 50):", minvalue=0, maxvalue=255)
        if threshold is None:
            return

        # Create a mask for segmentation
        mask = diff < threshold  # Create a mask based on the threshold

        # Create a segmented image
        segmented = np.zeros_like(img_np)  # Create an empty array for the segmented image
        segmented[mask] = img_np[mask]  # Keep the pixels that match the mask

        # Convert back to an image
        img_display = Image.fromarray(segmented)  # Convert the segmented array back to an image
        
        # Resize the image to fit the display
        img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)  # Resize the image

        # Update the canvas sizes and show the images
        update_canvas_sizes()
        show_images()  # Display the segmented image
    else:
        messagebox.showwarning("Warning", "No image or color selected for segmentation.")

def on_resize(event):
    """Adjust canvas sizes when the window is resized."""
    update_canvas_sizes()
    show_images()

def apply_filter(filter_type):
    """Applies the selected filter to the displayed image."""
    global img_display
    if filter_type == 'Sharpen':
        img_display = img_display.filter(ImageFilter.SHARPEN)
    elif filter_type == 'Smooth':
        img_display = img_display.filter(ImageFilter.SMOOTH)
    elif filter_type == 'Edge Detection':
        img_display = img_display.filter(ImageFilter.FIND_EDGES)
    elif filter_type == 'Emboss':
        img_display = img_display.filter(ImageFilter.EMBOSS)
    else:
        img_display = original_img.copy()
    show_images()
def adjust_intensity():
    global img_display
    factor = simpledialog.askfloat("Adjust Intensity", "Enter intensity factor:", minvalue=0.1, maxvalue=10.0)
    if factor:
        enhancer = ImageEnhance.Brightness(img_display)
        img_display = enhancer.enhance(factor)
        show_images()

def adjust_contrast():
    global img_display
    factor = simpledialog.askfloat("Adjust Contrast", "Enter contrast factor:", minvalue=0.1, maxvalue=10.0)
    if factor:
        enhancer = ImageEnhance.Contrast(img_display)
        img_display = enhancer.enhance(factor)
        show_images()

def adjust_color_balance():
    global img_display
    factor = simpledialog.askfloat("Adjust Color Balance", "Enter color balance factor:", minvalue=0.1, maxvalue=10.0)
    if factor:
        enhancer = ImageEnhance.Color(img_display)
        img_display = enhancer.enhance(factor)
        show_images()

def linear_transformation():
    global img_display
    factor = simpledialog.askfloat("Linear Transformation", "Enter linear transformation factor (k):", minvalue=0.1, maxvalue=5.0)
    if factor:
        img_display = img_display.point(lambda i: i * factor)
        show_images()

def log_transformation():
    global img_display
    # Convert to numpy array for pixel manipulation
    img_np = np.array(img_display)
    img_np = img_np.astype(np.float32)
    
    # Apply the logarithmic transformation
    c = 255 / np.log(1 + np.max(img_np))
    img_np = c * np.log(1 + img_np)
    
    # Convert back to an image
    img_np = np.uint8(img_np)
    img_display = Image.fromarray(img_np)
    
    show_images()

def power_law_transformation():
    global img_display
    gamma = simpledialog.askfloat("Power-Law Transformation", "Enter gamma value:", minvalue=0.1, maxvalue=5.0)
    if gamma:
        img_np = np.array(img_display)
        img_np = 255 * (img_np / 255) ** gamma
        img_np = np.uint8(img_np)
        img_display = Image.fromarray(img_np)
        show_images()

# Function for deep learning segmentation
def deep_learning_segment():
    global img_display
    if img_display:
        # Maintain the original size of the image and convert it to NumPy array
        original_size = img_display.size
        img_array = np.array(img_display) / 255.0  # Normalize the image to [0, 1]

        # Resize image for model input size (128x128) for segmentation prediction
        img_resized = img_display.resize((128, 128))
        img_array_resized = np.array(img_resized) / 255.0  # Normalize the image
        img_array_resized = img_array_resized.reshape(1, 128, 128, 3)  # Reshape for model input

        # Perform segmentation using the model
        prediction = model.predict(img_array_resized)  # Get predicted mask probabilities

        # Threshold the prediction to create the mask (binary)
        threshold = 0.05  # Set a lower threshold
        mask = (prediction > threshold).astype(np.uint8)[0, :, :, 0]  # Binary mask (128x128)

        # Resize the mask back to the original image size
        mask_resized = Image.fromarray(mask).resize(original_size, Image.Resampling.NEAREST)
        mask_resized_np = np.array(mask_resized)

        # Convert the mask to 3 channels for RGB (to match the image)
        mask_rgb = np.stack([mask_resized_np] * 3, axis=-1)  # Shape: (original_size, 3)

        # Create a light blue filter (RGB value for light blue) for the background
        light_blue_color = np.array([173, 216, 230], dtype=np.float32) / 255.0  # Light blue in [0, 1] range
        light_blue_filter = np.ones_like(img_array) * light_blue_color  # Light blue overlay filter

        # Choose a light coral color for the segmented object
        light_coral_color = np.array([240, 128, 128], dtype=np.float32) / 255.0  # Light coral in [0, 1] range
        light_coral_filter = np.ones_like(img_array) * light_coral_color  # Light coral overlay filter

        # Define alpha for blending
        alpha_bg = 0.3  # Transparency for the blue tint
        alpha_obj = 0.7  # Transparency for the light red color tint

        # Create the background with light blue tint
        background_with_blue = (1 - alpha_bg) * img_array + alpha_bg * light_blue_filter  # Blend blue and original

        # Create the segmented image: keep original colors where mask == 1, and apply light coral tint where mask == 1
        segmented_img = np.where(mask_rgb == 1, (1 - alpha_obj) * img_array + alpha_obj * light_coral_filter, background_with_blue)

        # Convert to image for display
        img_display = Image.fromarray((segmented_img * 255).astype(np.uint8))  # Convert to uint8 for proper display

        # Resize the image to fit the display canvas (final step for quality)
        img_display = img_display.resize((LARGE_DISPLAY_WIDTH, LARGE_DISPLAY_HEIGHT), Image.Resampling.LANCZOS)

        # Display the segmented image
        show_images()
    else:
        messagebox.showwarning("Warning", "No image available for segmentation.")







# Setup GUI window
root = tk.Tk()
root.title("PicEdit")

# Open the image file
logo_image = Image.open("C:/Users/User/Desktop/1sem/DIP/assignment/picedit-logo.jpg")

#Logo size
logo_image = logo_image.resize((50, 50))

# Convert the image to a format Tkinter can display
logo_image_tk = ImageTk.PhotoImage(logo_image)

# Create frames for the layout
frame_left = tk.Frame(root, width=200, bg="lightgrey")
frame_left.grid(row=0, column=0, sticky="ns")  # Fill vertically

frame_center = tk.Frame(root, bg="#b0c4de") #ash blue
frame_center.grid(row=0, column=1, sticky="nsew")  # Fill both vertically and horizontally

frame_right = tk.Frame(root, width=200, bg="lightgrey")
frame_right.grid(row=0, column=2, sticky="ns")  # Fill vertically

# Configure grid weight for responsive layout
root.grid_rowconfigure(0, weight=1)
root.grid_columnconfigure(1, weight=1)

# Create a frame to center the canvases
canvas_frame = tk.Frame(frame_center, bg="#b0c4de")
canvas_frame.grid(row=0, column=0, pady=40, padx=100)  # Add some padding for better spacing

# Create canvas for displaying original image with a light blue background
canvas_original = tk.Canvas(canvas_frame, width=LARGE_DISPLAY_WIDTH, height=LARGE_DISPLAY_HEIGHT, bg="lightblue")
canvas_original.grid(row=0, column=0, padx=20, pady=(0, 10))  # Add padding below

# Create label for original image directly under the canvas
label_original = tk.Label(canvas_frame, text="Original", bg="#b0c4de", font=("Helvetica", 12, "bold"))
label_original.grid(row=1, column=0, sticky="ew")  # Stretch to fill the width of the cell

# Create canvas for displaying updated image with a light blue background
canvas_updated = tk.Canvas(canvas_frame, width=LARGE_DISPLAY_WIDTH, height=LARGE_DISPLAY_HEIGHT, bg="lightblue")
canvas_updated.grid(row=0, column=1, padx=60, pady=(0, 10))  # Add padding below

# Create label for updated image directly under the canvas
label_updated = tk.Label(canvas_frame, text="Updated", bg="#b0c4de", font=("Helvetica", 12, "bold"))
label_updated.grid(row=1, column=1, sticky="ew")  # Stretch to fill the width of the cell

# Optionally, set the column weights to ensure proper sizing
canvas_frame.grid_columnconfigure(0, weight=1)  # Allow the first column to expand
canvas_frame.grid_columnconfigure(1, weight=1)  # Allow the second column to expand

# Bind mouse events to canvases
canvas_updated.bind("<ButtonPress-1>", start_crop)
canvas_updated.bind("<B1-Motion>", update_crop)
canvas_updated.bind("<ButtonRelease-1>", end_crop)
canvas_updated.bind("<Button-3>", pick_color)

# Create label for Basic Settings
label_basic_settings = tk.Label(frame_left, text="Basic Settings", bg="lightgrey", font=("Arial", 14, "bold"))
label_basic_settings.grid(row=0, column=0, pady=(10, 10), padx=(10, 10), sticky="ew")  # Add padding

# Define grey to black gradient color shades for buttons
button_colors = [
    "#A9A9A9",  # Light grey
    "#8F8F8F",  # Medium grey
    "#808080",  # Slightly darker grey
    "#696969",  # Dark grey
    "#505050",  # Darker grey, close to black
]

# Create buttons for the left frame with gradient colors
btn_open = tk.Button(frame_left, text="Open Image", command=open_image, bg=button_colors[0], fg="white")
btn_open.grid(row=1, column=0, pady=7, padx=10, sticky="ew")

btn_rotate = tk.Button(frame_left, text="Rotate", command=rotate_image, bg=button_colors[1], fg="white")
btn_rotate.grid(row=2, column=0, pady=7, padx=10, sticky="ew")

btn_flip_horizontal = tk.Button(frame_left, text="Flip Horizontal", command=lambda: flip_image('Horizontal'), bg=button_colors[2], fg="white")
btn_flip_horizontal.grid(row=3, column=0, pady=7, padx=10, sticky="ew")

btn_flip_vertical = tk.Button(frame_left, text="Flip Vertical", command=lambda: flip_image('Vertical'), bg=button_colors[3], fg="white")
btn_flip_vertical.grid(row=4, column=0, pady=7, padx=10, sticky="ew")

btn_reset = tk.Button(frame_left, text="Reset to Original", command=reset_image, bg=button_colors[4], fg="white")
btn_reset.grid(row=5, column=0, pady=7, padx=10, sticky="ew")

btn_save = tk.Button(frame_left, text="Save Image", command=save_image, bg=button_colors[0], fg="white")
btn_save.grid(row=6, column=0, pady=7, padx=10, sticky="ew")

label_crop = tk.Label(frame_left, text="(Left click and drag)", bg="lightgrey", font=("Arial", 10, "italic"))
label_crop.grid(row=7, column=0, pady=(5, 5), padx=(5, 5), sticky="ew")  

btn_save = tk.Button(frame_left, text="Crop Image", bg=button_colors[1], fg="white")
btn_save.grid(row=8, column=0, pady=7, padx=10, sticky="ew")

label_deep_settings = tk.Label(frame_left, text="Deep Learning", bg="lightgrey", font=("Arial", 14, "bold"))
label_deep_settings.grid(row=9, column=0, pady=(10, 10), padx=(10, 10), sticky="ew")  

btn_deep = tk.Button(frame_left, text="Semantic Segmentation", command=deep_learning_segment, bg=button_colors[2], fg="white")
btn_deep.grid(row=10, column=0, pady=7, padx=10, sticky="ew")

#Add the Image Enhancement Fuction Here
btn_deep = tk.Button(frame_left, text="Test", command= save_image , bg=button_colors[3], fg="white")
btn_deep.grid(row=11, column=0, pady=7, padx=10, sticky="ew")

label_crop = tk.Label(frame_left, text="Thank", bg="lightgrey", font=("Verdana", 12, "italic", "bold"), fg="#483D8B")
label_crop.grid(row=12, column=0, pady=(5, 5), padx=(5, 5), sticky="ew")  

label_crop = tk.Label(frame_left, text="You", bg="lightgrey", font=("Verdana", 12, "italic", "bold"),  fg="#483D8B")
label_crop.grid(row=13, column=0, pady=(5, 5), padx=(5, 5), sticky="ew")  

label_crop = tk.Label(frame_left, text="For", bg="lightgrey", font=("Verdana", 12, "italic", "bold"),  fg="#483D8B")
label_crop.grid(row=14, column=0, pady=(5, 5), padx=(5, 5), sticky="ew")  

label_crop = tk.Label(frame_left, text="Using", bg="lightgrey", font=("Verdana", 12, "italic", "bold"), fg="#483D8B")
label_crop.grid(row=15, column=0, pady=(5, 5), padx=(5, 5), sticky="ew")  

label_crop = tk.Label(frame_left, text="PicEdit", bg="lightgrey", font=("Verdana", 12, "italic", "bold"), fg="#483D8B")
label_crop.grid(row=16, column=0, pady=(5, 5), padx=(5, 5), sticky="ew")  

label_logo = tk.Label(frame_left, image=logo_image_tk, bg="lightgrey")
label_logo.grid(row=17, column=0, pady=(5, 5), padx=(5, 5), sticky="nsew")  
 



#Advanced Settings
label_advanced = tk.Label(frame_right, text="Advanced Settings", font=("Helvetica", 12, "bold"), bg="lightgrey")
label_advanced.grid(row=0, column=0, pady=(10, 10), padx=(10, 10))

# Advanced settings buttons with gradient colors
label_advanced = tk.Label(frame_right, text="(Right click the image to select color)", font=("Helvetica", 10, "italic"), bg="lightgrey")
label_advanced.grid(row=1, column=0, pady=(10, 10), padx=(5, 5))

btn_segment = tk.Button(frame_right, text="Segment Image", width=20, command=segment_image, bg=button_colors[1], fg="white")
btn_segment.grid(row=2, column=0, pady=7, padx=10, sticky="ew")

btn_intensity = tk.Button(frame_right, text="Adjust Intensity", command=adjust_intensity, bg=button_colors[2], fg="white")
btn_intensity.grid(row=3, column=0, pady=7, padx=10, sticky="ew")

btn_contrast = tk.Button(frame_right, text="Adjust Contrast", command=adjust_contrast, bg=button_colors[3], fg="white")
btn_contrast.grid(row=4, column=0, pady=7, padx=10, sticky="ew")

btn_color_balance = tk.Button(frame_right, text="Color Balance", command=adjust_color_balance, bg=button_colors[4], fg="white")
btn_color_balance.grid(row=5, column=0, pady=7, padx=10, sticky="ew")

btn_linear_transform = tk.Button(frame_right, text="Linear Transform", command=linear_transformation, bg=button_colors[0], fg="white")
btn_linear_transform.grid(row=6, column=0, pady=7, padx=10, sticky="ew")

btn_log_transform = tk.Button(frame_right, text="Log Transform", command=log_transformation, bg=button_colors[1], fg="white")
btn_log_transform.grid(row=7, column=0, pady=7, padx=10, sticky="ew")

btn_gamma_transform = tk.Button(frame_right, text="Gamma Transform", command=power_law_transformation, bg=button_colors[2], fg="white")
btn_gamma_transform.grid(row=8, column=0, pady=7, padx=10, sticky="ew")

# Filter buttons with gradient colors
btn_sharpen = tk.Button(frame_right, text="Sharpen", command=lambda: apply_filter('Sharpen'), bg=button_colors[3], fg="white")
btn_sharpen.grid(row=9, column=0, pady=7, padx=10, sticky="ew")

btn_smooth = tk.Button(frame_right, text="Smooth", command=lambda: apply_filter('Smooth'), bg=button_colors[4], fg="white")
btn_smooth.grid(row=10, column=0, pady=7, padx=10, sticky="ew")

btn_edge_detection = tk.Button(frame_right, text="Edge Detection", command=lambda: apply_filter('Edge Detection'), bg=button_colors[0], fg="white")
btn_edge_detection.grid(row=11, column=0, pady=7, padx=10, sticky="ew")

btn_emboss = tk.Button(frame_right, text="Emboss", command=lambda: apply_filter('Emboss'), bg=button_colors[1], fg="white")
btn_emboss.grid(row=12, column=0, pady=7, padx=10, sticky="ew")

label_advanced = tk.Label(frame_right, text="Image Color Conversion", font=("Helvetica", 10, "italic", "bold"), bg="lightgrey")
label_advanced.grid(row=13, column=0, pady=(10, 10), padx=(5, 5))

# Option menu for color conversion
color_option = tk.StringVar(value='Color')
color_menu = tk.OptionMenu(frame_right, color_option, 'Color', 'Grayscale', 'Black & White', command=convert_color)
color_menu.grid(row=14, column=0, pady=7, padx=10, sticky="ew")


# Bind resize event to adjust canvas sizes
root.bind("<Configure>", on_resize)

root.mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 652ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 441ms/step
